Based on the tutorial by Sean Robertson <a href="https://pytorch.org/tutorials/intermediate/char_rnn_generation_tutorial.html">here</a>.

In [1]:
import glob
import os
import string
import unicodedata
from io import open

In [31]:
DATA_DIR = '../../data'

In [32]:
all_letters = string.ascii_letters + " .,;'-'"
n_letters = len(all_letters) + 1 # + 1 for EOS

In [33]:
def find_files(path):
    return glob.glob(path)

In [40]:
def unicode_to_ascii(s):
    return ''.join(c for c in unicodedata.normalize('NFD', s) 
                   if unicodedata.category(c) != 'Mn' and c in all_letters)

In [41]:
def read_lines(filename):
    lines = open(filename, encoding='utf-8').read().strip().split('\n')
    return [unicode_to_ascii(line) for line in lines]

In [42]:
category_lines = {}
all_categories = []
for filename in find_files('%s/names/*.txt' % DATA_DIR):
    category = os.path.splitext(os.path.basename(filename))[0]
    all_categories.append(category)
    lines = read_lines(filename)
    category_lines[category] = lines
n_categories = len(all_categories)
if n_categories == 0:
    raise RuntimeError('Data not found')
print(f'# categories: {n_categories} {all_categories}')
print(unicode_to_ascii("O'Néàl"))

# categories: 18 ['Czech', 'German', 'Arabic', 'Japanese', 'Chinese', 'Vietnamese', 'Russian', 'French', 'Irish', 'English', 'Spanish', 'Greek', 'Italian', 'Portuguese', 'Scottish', 'Dutch', 'Korean', 'Polish']
O'Neal
